# Build GPT2

In [5]:
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F

## Overall tranformer structure

- We'll be implementing the right hand side (the decoder) that composes GPT-2: each unit on the right will be a block in our transformer.

![Transformer Architecture from "Attention Is All You Need" by Vaswani et al.](./assets/transformer.png)

- The configuration below is the configuration for the entire tranformer, with each layer *h* pertaining to one of the blocks. We want to replicate the following structure from a GPT-2 model in Huggingface Transformers:

![HF Transformer](./assets/hf_transformer.png)

- The code below is the skeleton on GPT2 config and main module that will allow us to replicate that structure:

In [ ]:
@dataclass
class GPTConfig:
    block_size: int = 256
    vocab_size: int = 65
    n_layer: int = 6
    n_head: int = 8
    n_embd: int = 384

class GPT(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config
        # With nn.ModuleDict() index into submodules just like a dictionary
        self.tranformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd)
            )
        )
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

- nn.ModuleDict allows you to index into submodules using keys, just like a dictionary.

- nn.ModuleList allows us to index into each individual layer using an index, just like with a list

## Transformer Block

- Now let's implement the Block,...

- Unlike the original GPT2 paper, establish a clean residual pathway by taking the layer norm of x and applying attention/multilayer perceptron layer to it *then* adding it to the input *x*.  Since addition allows for an adulterated gradient flow during backpropagation, this pre-layer norm configuration is the better than the post-layer norm configuration where the norm is applied after the addition. More formally, Xiong et al. (2020) have shown that if post-layer norm is used, a warm-up stage is needed to avoid training instability whereas if pre-layer norm is used, the gradients are well-behaved at initialization. See the difference between original (Post-LN) GPT-2 implementation and the 'corrected' pre-LN implementation used here:

![Source: "On Layer Normalization in the Transformer Architecture" by Xiong et al. 2020](./assets/pre_vs_post_layer_norm.png)

- Finally, onto the Block:

In [ ]:
class Block(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.config = config    
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

- Note again how the layer norm is applied *before* the addition to the residual stream.

- Andrej notes that attention is a communication operation, where tokens communicate with each other and aggregate information.   Thus attention can be thought of as a pooling function/weighted sum function/reduce operation.  On the other hand, the multilayer perceptron (MLP) is applied to each token individually, with no information exchanged between the tokens.  Thus attention is a *reduce* and MLP is the *map* operation and a transformer is a repeated application of MapReduce. 

## Multilayer Perceptron

- Briefly summarizing from Andrej's previous video (Let's build GPT: from scratch, in code, spelled out.), multilayer perceptron is implemented using a standard "bottleneck architecture" where the dimensions are first expanded to learn more complex representations, nonlinearity is applied to help the model learn more complex patterns, and finally the data is projected down again to keep the computational complexity in check.  

In [ ]:
class MLP(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, config.n_embd*4)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(config.n_embd*4, config.n_embd)
    def forward(self,x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x    

- GPT-2 used an approximate version of GeLU because at the time of GPT-2's creation, the erf function  was very slow in TensorFlow and GPT-2 and the approximate version was used.  Today there's no reason to use the approximate version but Andrej is using the tanh approximation for veracity.  
- Also, GeLU is better than ReLU due to dead neuron problem since a local gradient is always present as seen below:

![Source: https://pytorch.org/docs/stable/generated/torch.nn.GELU.html](./assets/gelu.png)

## Causal Self Attention

## References

Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, L., & Polosukhin, I. (2017). Attention Is All You Need.  arXiv preprint arXiv:1706.03762 . 

Xiong, R., Yang, Y., He, D., Zheng, K., Zheng, S., Xing, C., Zhang, H., Lan, Y., Wang, L., & Liu, T. (2020). On Layer Normalization in the Transformer Architecture. arXiv preprint arXiv:2002.04745.